# Collect demonstrations

Interactive notebook for collecting demonstrations

In [1]:
# %matplotlib auto

import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import socket
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

from robot.al5d_position_controller import PositionController
from robot.al5d_simulated_position_controller import SimulatedPositionController


from camera.camera_controller import CameraController
from demonstration import Demonstration


In [2]:
experiment = "demonstration_collector"
# picking up a collection specific name
hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    run = "collect_freeform_raven"
elif hostname == "szenes.local":
    run = "collect_no_robot"
elif hostname == "tredy2":
    run = "tredy2_collect_freeform"
    #run = "tredy2_automove_rwp6d_00"
else:
    run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_position_controller = Config().get_experiment(
    exp["exp_position_controller"], 
    exp["run_position_controller"])
exp_camera_controller = Config().get_experiment(
    exp["exp_camera_controller"], exp["run_camera_controller"])



Hostname is tredy2
***ExpRun**: Loading pointer config file:
	/home/lboloni/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-tredy2.yaml
***ExpRun**: Configuration for exp/run: demonstration_collector/tredy2_collect_freeform successfully loaded
***ExpRun**: Configuration for exp/run: demonstration/freeform successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config /home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/experiment_configs/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_controller_tredy2 successfully loaded


In [3]:
# This was here to debug things, because it was not finding one of the cameras
import cv2

# identifying which cameras work
for i in range(10):
    cap = cv2.VideoCapture(i)
    # FIXME: set for the specified one
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 256)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 256)
    cap.set(cv2.CAP_PROP_FPS, 10)

    ret, frame = cap.read()
    if ret:
        print(f"Camera {i} is working")
    else:
        print(f"Camera {i} not working")
    cap.release()

Camera 0 not working
Camera 1 not working


[ WARN:0@2.797] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@2.798] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@2.798] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@2.799] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


Camera 2 is working
Camera 3 is working
Camera 4 not working
Camera 5 not working
Camera 6 not working
Camera 7 not working
Camera 8 not working
Camera 9 not working


[ WARN:0@4.644] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video4): can't open camera by index
[ERROR:0@4.645] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.645] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video5): can't open camera by index
[ERROR:0@4.646] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.646] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video6): can't open camera by index
[ERROR:0@4.646] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.646] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video7): can't open camera by index
[ERROR:0@4.647] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4.647] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video8): can't open camera by index
[ERROR:0@4.648] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup C

In [ ]:
# starting the robot controller
if exp["available_robot"]:
    robot_controller = PositionController(exp_position_controller)
else:
    robot_controller = SimulatedPositionController(exp_position_controller)
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True


task_dir = exp_target.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

demonstration = Demonstration(exp_target, demoname, parse_old_style = False)
demonstration.metadata["cameras"] = sorted(camera_controller.capture_devs.keys())
demonstration.save_metadata()
# fixme need to add the cameras into the demonstration

***ExpRun**: Configuration for exp/run: robot_al5d/pulse_controller_00 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/angle_controller_00 successfully loaded
Try out the backup /dev/ttyUSB0
{'height': 5.0, 'distance': 5.0, 'heading': 0.0, 'wrist_angle': -45.0, 'wrist_rotation': 75.0, 'gripper': 100}
cap2 works
cap3 works
Ok, demonstration with go into
	 /home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2026_01_09__13_18_59
Saved demonstration metadata to
	/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-experiments/demonstration/freeform/2026_01_09__13_18_59/_metadata.yaml


: 

In [ ]:
# Performs the recording. The robot is controlled through the primary 
# controller. This is also used to terminate the collection
# The information is collected by the demonstration recorder

controller = exp["primary_controller"]
    

if controller == "xbox":
    # the gamepad is only available on some machines, and only on Linux
    available_gamepad = True
    try:
        from remote_control.gamepad_controller import GamepadController
    except ModuleNotFoundError:
        print("Approxeng module not found, cannot use gamepad")
        available_gamepad = False
    exp_gamepad_controller = Config().get_experiment(exp["exp_gamepad_controller"], exp["run_gamepad_controller"])
    gamepad_controller = GamepadController(exp_gamepad_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration,
        camera_controller=camera_controller, remote_control=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    exp_keyboard_controller = Config().get_experiment(exp["exp_keyboard_controller"], exp["run_keyboard_controller"])
    from remote_control.keyboard_controller import KeyboardController
    kb_controller = KeyboardController(exp_keyboard_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "widowx":
    from remote_control.widowx_controller import WidowXController
    exp_widowx_controller = Config().get_experiment(exp["exp_widowx_controller"], exp["run_widowx_controller"])    
    widowx_controller = WidowXController(exp_widowx_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=widowx_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    widowx_controller.demonstration_recorder = demo_recorder
    widowx_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "camera":
    # FIXME: future controller which is based on a camera and capturing human movement etc.
    pass

if controller == "automove":
    from remote_control.automove_controller import AutoMoveController
    exp_automove_controller = Config().get_experiment(exp["exp_automove_controller"], exp["run_automove_controller"])
    automove_controller = AutoMoveController(exp_automove_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    automove_controller.generate_waypoints()
    # program_controller.interactive_confirm = False
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=automove_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    automove_controller.demonstration_recorder = demo_recorder
    demonstration.save_metadata()
    automove_controller.control()



***ExpRun**: Experiment default config /home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/experiment_configs/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/rc_widowx_00 successfully loaded


[INFO] [1767982739.593157272] [interbotix_robot_manipulation]: Initialized InterbotixRobotNode!
[INFO] [1767982740.943209392] [interbotix_robot_manipulation]: 
	Robot Name: wx250s
	Robot Model: wx250s
[INFO] [1767982740.944076942] [interbotix_robot_manipulation]: Initialized InterbotixRobotXSCore!
[INFO] [1767982740.952000081] [interbotix_robot_manipulation]: 
	Arm Group Name: arm
	Moving Time: 2.00 seconds
	Acceleration Time: 0.30 seconds
	Drive Mode: Time-Based-Profile
[INFO] [1767982740.952566067] [interbotix_robot_manipulation]: Initialized InterbotixArmXSInterface!
[INFO] [1767982741.460775777] [interbotix_robot_manipulation]: 
	Gripper Name: gripper
	Gripper Pressure: 50.0%
[INFO] [1767982741.462027977] [interbotix_robot_manipulation]: Initialized InterbotixGripperXSInterface!
 height:5.00
 distance:5.00
 heading:-0.05
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:-0.05
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00



WidowX heading joint 0.0015339808305725455 current on al5d 0.0
WidowX heading joint 0.0015339808305725455 current on al5d -0.046019424917176366


 height:5.00
 distance:5.00
 heading:-0.05
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00

 height:5.00
 distance:5.00
 heading:-0.00
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00



WidowX heading joint 0.0015339808305725455 current on al5d -0.046019424917176366
WidowX heading joint 0.0 current on al5d -0.046019424917176366


...

WidowX heading joint 0.003067961661145091 current on al5d -0.09203884983435273
WidowX heading joint 0.003067961661145091 current on al5d -0.09203884983435273


 height:5.00
 distance:5.00
 heading:-0.09
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00



WidowX heading joint 0.003067961661145091 current on al5d -0.09203884983435273


 height:5.00
 distance:5.00
 heading:-0.09
 wrist_angle:-45.00
 wrist_rotation:75.00
 gripper:100.00



WidowX heading joint 0.003067961661145091 current on al5d -0.09203884983435273


In [ ]:
demonstration.save_metadata()
demonstration.move_to_video(delete_img_files=True)


In [ ]:
print(demonstration.metadata)

In [ ]:
exp